In [7]:
import pandas as pd
import numpy as np
#import caracteristicasDerivadas
#from caracteristicasDerivadas import H_entropy, proporcionVocalesConsonantes, posicionPrimerDigito
import  matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
from pandas_profiling import ProfileReport

import sklearn
from sklearn import metrics, model_selection, tree

In [8]:
def H_entropy (x):
    # Entropia de Shannon
    prob = [ float(x.count(c)) / len(x) for c in dict.fromkeys(list(x)) ] 
    H = - sum([ p * np.log2(p) for p in prob ]) 
    return H

def posicionPrimerDigito( s ):
    for i, c in enumerate(s):
        if c.isdigit():
            return i + 1
    return 0

def proporcionVocalesConsonantes (x):
    x = x.lower()
    patron_vocales = re.compile('([aeiou])')
    patron_consonantes = re.compile('([b-df-hj-np-tv-z])')
    vocales = re.findall(patron_vocales, x)
    consonantes = re.findall(patron_consonantes, x)
    try:
        proporcion = len(vocales) / len(consonantes)
    except: # Division por cero
        proporcion = 0  
    return proporcion

## Exploración de datos

#### Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones. 

In [10]:
df = pd.read_csv("dataset_pishing.csv")

In [11]:
df.head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 67 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   url                         11430 non-null  object 
 1   ip                          11430 non-null  int64  
 2   nb_www                      11430 non-null  int64  
 3   nb_com                      11430 non-null  int64  
 4   nb_dslash                   11430 non-null  int64  
 5   http_in_path                11430 non-null  int64  
 6   punycode                    11430 non-null  int64  
 7   port                        11430 non-null  int64  
 8   tld_in_path                 11430 non-null  int64  
 9   tld_in_subdomain            11430 non-null  int64  
 10  abnormal_subdomain          11430 non-null  int64  
 11  nb_subdomains               11430 non-null  int64  
 12  prefix_suffix               11430 non-null  int64  
 13  random_domain               114

#### Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como “pishing”. ¿Está balanceado el dataset? 

In [13]:
df['status'].value_counts()

legitimate    5715
phishing      5715
Name: status, dtype: int64

Como se puede observar, sí se encuentra balanceado el dataset entre legit y phishing, dado que al contar sus valores se mantienen mitad/mitad

## Derivación de características 

In [14]:
df['f1']=df.apply(lambda row: len(row['url']), axis=1)
df['f2']=df.apply(lambda row: len(row['url'].split("//")[-1].split("/")[0].split('?')[0]), axis=1)
chars = ['.', '-', '@', '?', '&', '|', '=', '_', '~', '%', '/', '*', ':', ',', ';', '$', ' ']
for i in range(len(chars)):
    df[f'f{i+4}']=df.apply(lambda row: row['url'].count(chars[i]), axis=1)
df['f25']=df.apply(lambda row: 1 if row['url'].split(':')[0] == 'https' else 0, axis=1)
df['f26']=df.apply(lambda row: sum(c.isdigit() for c in row['url']), axis=1)
df['f27']=df.apply(lambda row: sum(c.isdigit() for c in row['url'].split("//")[-1].split("/")[0].split('?')[0]), axis=1)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 89 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   url                         11430 non-null  object 
 1   ip                          11430 non-null  int64  
 2   nb_www                      11430 non-null  int64  
 3   nb_com                      11430 non-null  int64  
 4   nb_dslash                   11430 non-null  int64  
 5   http_in_path                11430 non-null  int64  
 6   punycode                    11430 non-null  int64  
 7   port                        11430 non-null  int64  
 8   tld_in_path                 11430 non-null  int64  
 9   tld_in_subdomain            11430 non-null  int64  
 10  abnormal_subdomain          11430 non-null  int64  
 11  nb_subdomains               11430 non-null  int64  
 12  prefix_suffix               11430 non-null  int64  
 13  random_domain               114

In [16]:
df.head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,f14,f15,f16,f17,f18,f19,f20,f25,f26,f27
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,0,0,0
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,0,17,0
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,5,0,1,0,0,0,0,1,19,0
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,2,0,1,0,0,0,0,0,0,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,0,0,0


Se puede observar 5 observaciones del dataset al derivar las características, y se ve que se generaron nuevas columnas, de f1-f27, por lo cual fue efectivo el procesamiento.

## Preprocesamiento 

In [17]:
df['status'] = df.apply(lambda row: 0 if row['status'] == 'legitimate' else 1, axis=1)

In [19]:
df['status']

0        0
1        1
2        1
3        0
4        0
        ..
11425    0
11426    1
11427    0
11428    0
11429    1
Name: status, Length: 11430, dtype: int64

Como se puede ver se realizó el cambio de variable categórica a binaria de forma efectiva

## Visualización de resultados 

In [20]:
report = ProfileReport(df, title="Detección de phishing", explorative=True)
report.to_file("reporte.html")

Summarize dataset:   0%|          | 0/102 [00:00<?, ?it/s]

C:\Users\maria\AppData\Roaming\Python\Python37\site-packages\pandas_profiling\model\correlations.py:152: UserWarning: There was an attempt to calculate the spearman correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"spearman": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'Buffer dtype mismatch, expected 'const intp_t' but got 'long long'')
  (include the error message: '{error}')"""
C:\Users\maria\AppData\Roaming\Python\Python37\site-packages\pandas_profiling\model\correlations.py:152: UserWarning: There was an attempt to calculate the kendall correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"kendall": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com

MemoryError: In RendererAgg: Out of memory

MemoryError: In RendererAgg: Out of memory

<Figure size 576x396 with 1 Axes>

## Selección de Características 

In [21]:
df_lean = df.drop(['nb_com'
,'nb_dslash'
,'http_in_path'
,'punycode'
,'port'
,'tld_in_path'
,'tld_in_subdomain'
,'abnormal_subdomain'
,'path_extension'
,'nb_external_redirection'
,'brand_in_subdomain'
,'brand_in_path'
,'suspecious_tld'
,'statistical_report'
,'ratio_nullHyperlinks'
,'ratio_intRedirection'
,'ratio_intErrors'
,'submit_email'
,'sfh'
,'iframe'
,'popup_window'
,'onmouseover'
,'right_clic'
,'dns_record'], axis=1)

MemoryError: Unable to allocate 4.62 MiB for an array with shape (53, 11430) and data type int64

In [22]:
df_lean.info()

NameError: name 'df_lean' is not defined